# Importing Required Libraries

In [4]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, LSTM, MaxPooling1D
from keras.utils import to_categorical
from matplotlib import pyplot

# Loaing the File

In [13]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + '../UCI HAR Dataset/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + '../UCI HAR Dataset/')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

# Training the Model with Sequential Model

In [14]:
# fit and evaluate an LSTM model (do 10-fold cross validation)
def evaluate_model(trainX, trainy, testX, testy):
    # define model
    verbose, epochs, batch_size = 0, 15, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    print(n_timesteps, n_features, n_outputs)
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

# Training the Model with LSTM

In [20]:
def evaluate_lstm_model(trainX, trainy, testX, testy, n_neurons):
    # define model
    verbose, epochs, batch_size = 0, 15, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

    model = Sequential()
    model.add(LSTM(units=n_neurons, input_shape=(n_timesteps, n_features)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

# Summarizing the Results Obtained

In [15]:
# summarize scores 
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [17]:
# run an experiment
def run_experiment(repeats=10, n_neurons=100):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_lstm_model(trainX, trainy, testX, testy, n_neurons)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [19]:
# run the experiment
run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 89.922
>#2: 89.515
>#3: 90.974
>#4: 90.058
>#5: 90.431
>#6: 89.277
>#7: 90.126
>#8: 89.413
>#9: 88.972
>#10: 89.209
[89.92195725440979, 89.51476216316223, 90.97387194633484, 90.05768299102783, 90.43094515800476, 89.27723169326782, 90.12554883956909, 89.41296339035034, 88.97183537483215, 89.20936584472656]
Accuracy: 89.790% (+/-0.592)


# Testing Different Number of Neurons

### Number of Neurons = 100

In [21]:
def run_experiment(repeats=5, n_neurons=100):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_lstm_model(trainX, trainy, testX, testy, n_neurons)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 91.211
>#2: 89.956
>#3: 89.854
>#4: 91.177
>#5: 89.752
[91.21140241622925, 89.95589017868042, 89.85409140586853, 91.17746949195862, 89.75229263305664]
Accuracy: 90.390% (+/-0.660)


### Number of Neurons = 10

In [22]:
def run_experiment(repeats=5, n_neurons=10):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_lstm_model(trainX, trainy, testX, testy, n_neurons)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 81.642
>#2: 82.966
>#3: 87.852
>#4: 85.307
>#5: 84.934
[81.64234757423401, 82.9657256603241, 87.85205483436584, 85.30709147453308, 84.93382930755615]
Accuracy: 84.540% (+/-2.125)


### Number of Neurons = 200

In [23]:
def run_experiment(repeats=5, n_neurons=200):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_lstm_model(trainX, trainy, testX, testy, n_neurons)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 89.786
>#2: 88.225
>#3: 88.700
>#4: 90.431
>#5: 90.635
[89.78622555732727, 88.2253110408783, 88.70037198066711, 90.43094515800476, 90.63454270362854]
Accuracy: 89.555% (+/-0.947)


# Subjective Questions

**1. Why was this problem suitable for LSTM and nor just any other machine learning algorithm?**
- This problem is suitable for LSTM (Long Short-Term Memory) and not just any other machine learning algorithm due to the temporal nature of the data. The data involves sequences of activities over time, and LSTM is particularly well-suited for handling sequences and capturing long-term dependencies. In this case, the LSTM can learn and remember patterns in the sequence of movement data, which is crucial for recognizing different activities that unfold over time. Traditional machine learning algorithms are not as effective in capturing temporal aspects of the data.

**2. How do changing number of neurons in LSTM affect its performance?**
- Increasing the number of neurons in the LSTM increases the model's capacity to learn more complex patterns in the data. However, this also increases the risk of overfitting the training data. On the other hand, decreasing the number of neurons in the LSTM decreases the model's capacity to learn complex patterns in the data. This reduces the risk of overfitting the training data, but it also increases the risk of underfitting the training data. Therefore, the number of neurons in the LSTM is a hyperparameter that should be chosen carefully to balance the trade-off between model capacity and overfitting/underfitting. For instance, in this code, when I used 100 neurons, the accuracy of the model ranged from 89 to 91% approximately. However, when I used 10 neurons, the accuracy of the model ranged from 81 to 84% approximately. Finally, when I used 200 neurons, the accuracy was only about 89% on average despite the increased complexity of the model.  

**3. Is a many-to-many model a real-world scenario for this problem or a many-to-one model?**
- A many-to-one model is more appropriate. The input sequences (sensor data over time) are mapped to a single output (activity label). The goal is to classify the entire sequence of movement data into one of the six activities. A many-to-many model might be more suitable if the task involved predicting a sequence of labels for each time step, which is not the case here. 